In [10]:
! pip install -U segmentation-models
! pip install q keras==2.3.1
! pip install tensorflow==2.1.0

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing import image
import segmentation_models as sm
from skimage import measure
from skimage import filters
from scipy.spatial import cKDTree
from scipy import inf
import pandas as pd 

#### Count number of buildings around coordinates using OSM
Given a list of place locations and building locations, determine the number of buildings within a defined radius of each location. Uses **k-d tree** to implement nearest-neighbor lookup. 

In [ ]:
def count_osm_buildings(places_path, buildings_path, out_path):
  places = pd.read_csv(places_path)
  buildings = pd.read_csv(buildings_path)

  subset = places[['x', 'y']]
  places = [tuple(x) for x in subset.to_numpy()]

  subset = buildings[['x', 'y']]
  buildings = [tuple(x) for x in subset.to_numpy()]

  point_tree = spatial.cKDTree(buildings)
  df_building_count = pd.DataFrame(columns=['long', 'lat', 'building-count'])
  for place in places:
    df_building_count.loc[len(df_building_count)] = [place[0], place[1], len(point_tree.query_ball_point(place, 0.0045))]

  print(df_building_count)

  df_building_count.to_csv(out_path)

#### Count number of buildings from binary segmentation mask
Use **connected components** to count the number of buildings predicted within an image tile mask. 

In [2]:
def count_satellite_buildings(model_path, im_path):
  # load model 
  dependencies = {
      'iou_score': sm.metrics.iou_score,
      'precision': sm.metrics.precision,
      'recall': sm.metrics.recall
  }

  model_load = keras.models.load_model(model_path, custom_objects=dependencies)

  # load sample image
  img = image.load_img(im_path, target_size=(512, 512))
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)

  mask = model_load.predict(x).round()
  im_mask = image.array_to_img(mask[0])

  im = np.array(im_mask)
  all_labels = measure.label(im)
  b_count = measure.label(im, background=0, connectivity=2, return_num=True)

  return b_count[1]